# Tweeepy Ego-Network
### A 1-step distance network graph with weighted nodes and edges

Imports necessary for networkx to work.

In [2]:
import networkx as nx
import tweepy
import numpy as np
import random
from IPython.display import Image
import matplotlib.pyplot as plt

Authentication process for Twitter account. 

In [ ]:
auth = tweepy.OAuthHandler("XXXX", "XXXX")
auth.set_access_token("XXXX","XXXX")
api = tweepy.API(auth, wait_on_rate_limit=True)

Set root user (center of the graph, initial node)

In [ ]:
# root_user = api.me().screen_name if authenticated user is wanted as root
root_user = 'airbnb_mx'
u = api.get_user(root_user) # User api object

Initialize the graph. Define the function to set weights of edges. Set weights for every friend of the root user and for their friends, only taking the ones that are also friends with the root user.

In [ ]:
graph = nx.Graph()

# Add root_user to graph
graph.add_node(root_user, label=root_user, weight=u.statuses_count)
friends = api.friends(screen_name=root_user)

# Edge weight based on influence of following node A(the one who follows the other) and followers of A and B
def calculateEdgeWeight(follower_count_a, following_count_a, follower_count_b):
    follower_following_ratio = follower_count_a / following_count_a
    return follower_following_ratio * (follower_count_a - follower_count_b) / (follower_count_a + follower_count_b)

# Asigning weights
for f in friends:
    print("\nProcessing friends of ", f.screen_name)
    graph.add_node(f.screen_name, label=f.screen_name, weight=f.statuses_count)
    graph.add_edge(root_user, f.screen_name,
                   weight=calculateEdgeWeight(u.followers_count, u.friends_count, f.followers_count))
    if not f.protected:
        following_of_f = api.friends(screen_name=f.screen_name)
        for ff in following_of_f:
            for friend in friends:
                if friend.screen_name == ff.screen_name:
                    print(ff.screen_name, end=" | ")
                    graph.add_node(ff.screen_name, label=ff.screen_name, weight=ff.statuses_count)
                    graph.add_edge(f.screen_name, ff.screen_name, weight=calculateEdgeWeight(
                        f.followers_count, f.friends_count, ff.followers_count))

nx.read_gexf() can be used to read graphs outputed by networkx in an xml file. If the graph was already calculated it can be read uncommenting that line

In [ ]:
# graph = nx.read_gexf('path_to_gexf_file') # Read the graph if there already exits one
# design parameters
pos = nx.circular_layout(graph)
plt.figure(1,figsize=(10,10))

To start drawing the graph, we need to iterate through all the weights and asign widths to unique weights so that each edge and node has its own width/size. 

In [ ]:
all_weights = []
# Iterate through the graph nodes to gather all the weights
for (node1,node2,data) in graph.edges(data=True):
    all_weights.append(data['weight']) #we'll use this when determining edge thickness

max_edge_weight = max(all_weights)
# Get unique weights
unique_weights = list(set(all_weights))

# Plot the edges assigning specific edge widths to weights
for weight in unique_weights:
    # Form a filtered list with just the weight you want to draw
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in graph.edges(data=True) if edge_attr['weight']==weight]
    width = weight/max_edge_weight
    nx.draw_networkx_edges(graph,pos,edgelist=weighted_edges,width=width)

all_node_weights=[]
for node in graph.nodes(data=True):
    all_node_weights.append(node[1]['weight'])

node_sizes = []
max_node_weight = max(all_node_weights)
for node_weight in all_node_weights:
    node_sizes.append(node_weight/max_node_weight*300)
    
nx.draw_networkx_nodes(graph, pos, node_size=node_sizes)
nx.draw_networkx_labels(graph, pos)


Ploting the graph and setting the name to save it.

In [ ]:
#Plot the graph
plt.axis('off')
plt.title(root_user + '1-step Ego-Network')
plt.savefig("egoNetwork.png") 
plt.show()